In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html

import torch
import torch.nn.functional as F
import tqdm

from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader, LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T

dataset = TUDataset(root='/tmp/NCI1', name='NCI1', transform=T.NormalizeFeatures())
torch.manual_seed(12315)
dataset = dataset.shuffle()
dataset_length = len(dataset)

# split dataset into 3 parts
DA_train = dataset[0:int(0.4*dataset_length)]
D_aux = dataset[int(0.4*dataset_length):int(0.7*dataset_length)]
DA_test = dataset[int(0.7*dataset_length):]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html
     |████████████████████████████████| 8.0 MB 52.8 MB/s 
     |████████████████████████████████| 3.5 MB 54.1 MB/s 
     |████████████████████████████████| 2.4 MB 62.3 MB/s 
     |████████████████████████████████| 706 kB 62.0 MB/s 
     |████████████████████████████████| 467 kB 32.9 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689857 sha256=e3f302ee5f7c2d7e032189fcab0b2611d6c502656c3e5dbe3c41cbc8550bb4c6
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


Extracting /tmp/NCI1/NCI1/NCI1.zip
Processing...
Done!


In [2]:
print(len(DA_train))
DA_train_train = DA_train[0:int(0.7*len(DA_train))]
DA_train_val = DA_train[int(0.7*len(DA_train)):]
print(len(DA_train_train))
print(len(DA_train_val))

1644
1150
494


In [3]:
len(D_aux)

1233

In [4]:
len(DA_test)

1233

In [5]:
train_data = Batch.from_data_list(DA_train_train)
test_data = Batch.from_data_list(DA_train_val)

train_loader = LinkNeighborLoader(train_data, batch_size=2048, shuffle=True,
                            neg_sampling_ratio=0.5, num_neighbors=[10,10],
                            num_workers=2, persistent_workers=True)

test_loader = LinkNeighborLoader(test_data, batch_size=2048, shuffle=False,
                            neg_sampling_ratio=0.5, num_neighbors=[10,10],
                            num_workers=2, persistent_workers=True)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(
            in_channels=DA_train.num_features,
            hidden_channels=192*2,
            num_layers=3,
            out_channels=192
        ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [7]:
def train():
    model.train()
    
    total_loss = total_examples = 0
    
    for data in tqdm.tqdm(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        h = model(data.x, data.edge_index)
        
        h_src = h[data.edge_label_index[0]]
        h_dst = h[data.edge_label_index[1]]
        
        link_pred = (h_src * h_dst).sum(dim=-1)
        
        loss = F.binary_cross_entropy_with_logits(link_pred, data.edge_label)
        
        loss.backward()
        optimizer.step()
        
        total_loss += float(loss) * link_pred.numel()
        total_examples += link_pred.numel()
        
    return total_loss / total_examples

In [8]:
def test():
    model.eval()
    
    total_loss = total_examples = 0
    
    for data in tqdm.tqdm(test_loader):
        data = data.to(device)
        # optimizer.zero_grad()
        
        h = model(data.x, data.edge_index)
        
        h_src = h[data.edge_label_index[0]]
        h_dst = h[data.edge_label_index[1]]
        
        link_pred = (h_src * h_dst).sum(dim=-1)
        
        loss = F.binary_cross_entropy_with_logits(link_pred, data.edge_label)
        
#         loss.backward()
#         optimizer.step()
        
        total_loss += float(loss) * link_pred.numel()
        total_examples += link_pred.numel()
        
    return total_loss / total_examples

In [9]:
for epoch in range(1,100):
    train_loss = train()
    test_loss = test()
    print(f'Epoch: {epoch:02d}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}')

100%|██████████| 16/16 [00:00<00:00, 27.62it/s]


Epoch: 01, Train loss: 0.6650, Test loss: 0.4568


100%|██████████| 16/16 [00:00<00:00, 31.45it/s]


Epoch: 02, Train loss: 0.4460, Test loss: 0.4176


100%|██████████| 16/16 [00:00<00:00, 34.20it/s]


...

Epoch: 97, Train loss: 0.2939, Test loss: 0.2865


100%|██████████| 16/16 [00:00<00:00, 31.98it/s]


Epoch: 98, Train loss: 0.2922, Test loss: 0.2771


100%|██████████| 16/16 [00:00<00:00, 18.83it/s]

Epoch: 99, Train loss: 0.2923, Test loss: 0.2803


# save the model to local

In [10]:
data_save_path = "NCI_model.pt"
torch.save(model, data_save_path)